# Flux Architecture for managing globalstates
## Key Components
- Actions: These are payloads of information that send data from the application to the store. They are the only source of information for the store.
- Dispatcher: This acts as a central hub that manages all data flow in the application. It receives actions and broadcasts payloads to registered callbacks.
- Stores: These contain the application state and logic. They are similar to models in a traditional MVC pattern, but they manage the state of many objects rather than a single object's data.
- Views: These are typically React components that listen to changes from the stores and re-render themselves accordingly.
## Data flow 
### User action -> Dispatchers -> stores -> Views
- User interacts with a View.
- View propagates an Action triggered by the user.
- Dispatcher receives the Action and broadcasts it to the appropriate Store(s).
- Store(s) update their internal state based on the Action.
- View(s) are notified of the state change and re-render with the new data.
## Implementations
- Redux: A popular implementation of Flux that introduces additional concepts like reducers and a single store.
- MobX: An alternative to Flux that uses observable data structures.
- Vuex: A Flux-inspired state management pattern for Vue.js applications.
- NgRx: A reactive state management library for Angular applications, inspired by Redux.

In [ ]:
from typing import Callable, Dict, List
from abc import ABC, abstractmethod

# Dispatcher
class Dispatcher:
    def __init__(self):
        self.callbacks: List[Callable] = []

    def register(self, callback: Callable):
        self.callbacks.append(callback)

    def dispatch(self, action: Dict):
        for callback in self.callbacks:
            callback(action)

# Action Types
INCREMENT = 'INCREMENT'
DECREMENT = 'DECREMENT'

# Action Creators
def increment_action():
    return {'type': INCREMENT}

def decrement_action():
    return {'type': DECREMENT}

# Store
class Store(ABC):
    @abstractmethod
    def get_state(self):
        pass

    @abstractmethod
    def add_listener(self, listener: Callable):
        pass

class CounterStore(Store):
    def __init__(self, dispatcher: Dispatcher):
        self.count = 0
        self.listeners: List[Callable] = []

        dispatcher.register(self.handle_action)

    def handle_action(self, action: Dict):
        if action['type'] == INCREMENT:
            self.count += 1
        elif action['type'] == DECREMENT:
            self.count -= 1
        self.emit_change()

    def get_state(self):
        return self.count

    def add_listener(self, listener: Callable):
        self.listeners.append(listener)

    def emit_change(self):
        for listener in self.listeners:
            listener()

# View (simulated)
class CounterView:
    def __init__(self, store: Store):
        self.store = store
        store.add_listener(self.render)

    def render(self):
        print(f"Count: {self.store.get_state()}")

# Usage
if __name__ == "__main__":
    dispatcher = Dispatcher()
    store = CounterStore(dispatcher)
    view = CounterView(store)

    print("Initial state:")
    view.render()

    print("\nIncrementing:")
    dispatcher.dispatch(increment_action())

    print("\nIncrementing again:")
    dispatcher.dispatch(increment_action())

    print("\nDecrementing:")
    dispatcher.dispatch(decrement_action())